In [1]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss

C:\anaconda3\envs\tf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\anaconda3\envs\tf\lib\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
C:\anaconda3\envs\tf\lib\site-packages\torchvision\transforms\v2\__init__.py:54: UserWarning: The to

In [3]:
df = pd.read_csv("../dataset/Balanced_Conversation_Data.csv")  # Make sure it has 'text' and 'label' columns
df = df.dropna(subset=["text", "label"])

texts = df['text'].tolist()
labels = df['label'].tolist()

In [4]:
model = SentenceTransformer('all-MiniLM-L6-v2')  # Light and fast
embeddings = model.encode(texts, convert_to_numpy=True, show_progress_bar=True)

Batches: 100%|███████████████████████████████████████████████████████████████████| 22214/22214 [21:23<00:00, 17.30it/s]


In [5]:
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)  # Add the dataset embeddings to FAISS

In [6]:
def predict_label(input_text, top_k=1):
    input_embed = model.encode([input_text], convert_to_numpy=True)
    distances, indices = index.search(input_embed, top_k)

    results = []
    for i in range(top_k):
        idx = indices[0][i]
        sim_score = 1 - distances[0][i] / 4  # Cosine approx (optional for interpretation)
        results.append((texts[idx], labels[idx], sim_score))
    
    predicted_label = results[0][1]
    print(f"\nInput: {input_text}")
    print(f"Top Match: {results[0][0]}")
    print(f"Predicted Label: {predicted_label} (Confidence~{results[0][2]:.2f})")
    return predicted_label

In [8]:
predict_label("I feel hopeless and don't enjoy anything anymore")
predict_label("I get really scared and my heart races even when nothing is wrong")
predict_label("I feel like I should end my life because there is so much of disturbance around me and I feel hopeless")


Input: I feel hopeless and don't enjoy anything anymore
Top Match: I feel like I'm constantly feeling hopeless.
Predicted Label: depression (Confidence~0.89)

Input: I get really scared and my heart races even when nothing is wrong
Top Match: Every time I try to relax, my heart starts racing, and I get really anxious for no reason.
Predicted Label: anxiety (Confidence~0.85)

Input: I feel like I should end my life because there is so much of disturbance around me and I feel hopeless
Top Match: I just want to end it alli cant change the way i feel no matter what i do in life. it always just seems the world would be a better place without me
Predicted Label: suicide (Confidence~0.87)


'suicide'

In [9]:
predict_label("Thank you for being here… honestly, I don’t even know where to start. Everything just feels so heavy all the time. It’s like I’m constantly trying to stay afloat, but the more I try, the more I feel like I’m sinking. I don’t see the point in anything anymore… and I’m so tired. I'm sorry if that sounds dramatic — I just don’t know what else to do.")


Input: Thank you for being here… honestly, I don’t even know where to start. Everything just feels so heavy all the time. It’s like I’m constantly trying to stay afloat, but the more I try, the more I feel like I’m sinking. I don’t see the point in anything anymore… and I’m so tired. I'm sorry if that sounds dramatic — I just don’t know what else to do.
Top Match: I’m tired.I’ve worked so hard to get to the place that I am now, and still feel like it will never be enough. I feel like since I was young that I knew I was going to “go out by my own hand.” It might sound crazy but I feel like somewhere deep down I’ve always felt this way. I’m so tired of feeling anxious to the point of being physically sick. I’m tired of the contest barrage of thoughts. They’re always going. I can’t get my mind to stop. I just want it to stop. I just want to be happy, for once. I want to feel genuinely happy. I want to feel something other than wanting to die. I feel like everyday I’m drowning. Everyday I

'suicide'

In [10]:
predict_label("Yeah… I guess it’s not just one thing — it’s everything piling up. I feel like I keep disappointing the people around me. No matter how hard I try, it's never enough. At school, at home… it’s like I’m invisible until I mess something up. Then suddenly everyone notices — and not in a good way.")


Input: Yeah… I guess it’s not just one thing — it’s everything piling up. I feel like I keep disappointing the people around me. No matter how hard I try, it's never enough. At school, at home… it’s like I’m invisible until I mess something up. Then suddenly everyone notices — and not in a good way.
Top Match: I feel like I'm constantly disappointing the people around me.
Predicted Label: stress (Confidence~0.85)


'stress'

In [11]:
np.save("embeddings.npy", embeddings)
df.to_csv("balanced_cleaned_dataset.csv", index=False)